In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/app')

In [3]:

# %%
from qdrant_client import QdrantClient
from datetime import datetime, timedelta
import pandas as pd

In [4]:

pd.set_option('display.max_colwidth', None)

# %% [markdown]
# ## Connect to Qdrant

# %%
def get_qdrant_client():
    """Get Qdrant client with default settings"""
    return QdrantClient(
        host="qdrant",  # Update if running in different environment
        port=6333
    )

# %% [markdown]
# ## Query Recent Chunks

# %%
def get_recent_chunks(collection_name="meeting_chunks", limit=100, hours_ago=24*30):
    """
    Retrieve the most recent chunks from Qdrant.
    
    Args:
        collection_name (str): Name of the Qdrant collection
        limit (int): Maximum number of chunks to retrieve
        hours_ago (int): Look back period in hours
    
    Returns:
        pd.DataFrame: DataFrame containing chunk data
    """
    client = get_qdrant_client()
    
    # Calculate timestamp threshold
    time_threshold = (datetime.utcnow() - timedelta(hours=hours_ago)).isoformat()
    
    # Search with filtering by timestamp
    search_result = client.scroll(
        collection_name=collection_name,
        scroll_filter={"must": [
            {"key": "timestamp", "range": {"gte": time_threshold}}
        ]},
        limit=limit,
        with_payload=True,
        with_vectors=True
    )[0]  # scroll returns (points, next_page_offset)
    
    # Extract relevant fields from payload
    chunks_data = []
    for point in search_result:
        payload = point.payload
        chunks_data.append({
            'id': point.id,
            'timestamp': payload.get('timestamp'),
            'meeting_id': payload.get('meeting_id'),
            'content': payload.get('content'),
            'contextualized_content': payload.get('contextualized_content'),
            'chunk_index': payload.get('chunk_index'),
            'topic': payload.get('topic'),
            'speaker': payload.get('speaker'),
            'speakers': payload.get('speakers'),
            'vector_dim': len(point.vector) if point.vector else None
        })
    
    # Convert to DataFrame
    df = pd.DataFrame(chunks_data)
    
    return df

In [5]:
get_recent_chunks().sort_values('timestamp', ascending=False)

id                         timestamp  \
83  0f24c46c-b697-40c4-95bf-50648921d31f        2024-12-27T16:13:36.933204   
80  0eecc3d0-4987-485b-a678-9677f257f8a1        2024-12-26T20:44:32.201285   
77  0ec8273c-0539-4377-abe9-238607cbe759  2024-12-20T18:01:44.440000+00:00   
82  0f1f3858-c6af-4ce8-a576-af7362b34d93  2024-12-20T13:25:46.510000+00:00   
20  03c13257-988b-4898-8360-3c933cd8b5de  2024-12-20T13:25:46.510000+00:00   
..                                   ...                               ...   
25  04f8181c-d77c-4628-bc8f-3172cc181dff  2024-12-02T15:30:37.920000+00:00   
9   023a0f91-f520-4e14-a871-723a78d8cd6d  2024-12-02T15:30:37.920000+00:00   
55  09361439-5c49-4309-a4ea-9696b28fe4fe  2024-12-02T12:10:23.270000+00:00   
35  063698cf-7950-4cd0-b768-6a6d87c5190f  2024-11-28T13:12:07.530000+00:00   
81  0f10b7dd-78e9-44e2-8db1-814ff0c2c2a7  2024-11-28T13:12:07.530000+00:00   

                              meeting_id  \
83  71a99bf5-2ea9-4e51-815d-acd0e56ded50   
80  0d77eb5b-6fa1-453e-ad6a-c7fa65f2f278   
77  f3eb1b39-c725-4ecf-8c4d-bd78eac81273   
82  794b68e5-0dc5-4cb4-b221-c42e338b6743   
20  794b68e5-0dc5-4cb4-b221-c42e338b6743   
..                                   ...   
25  b2df54b2-e91e-4d41-bd4a-48c8fc99e5fb   
9   b2df54b2-e91e-4d41-bd4a-48c8fc99e5fb   
55  1579143f-3d96-4346-9220-febb329ca5ce   
35  16705c46-ba51-4f1b-83d2-08e0bfc5eae5   
81  16705c46-ba51-4f1b-83d2-08e0bfc5eae5   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            